---
title: "Monte Carlo Simulations for Monte Carlo Games"
date: 2023-04-21T12:47:26-04:00
draft: false
tags:
- "monte carlo"
summary: "A Neovim GUI colorscheme with blocky highlighting"
---

This article is an attempt to use the Monte Carlo method to simulate a game of roulette. 
The method was inspired by a roulette table at the famous casino, so it seemed fitting to apply the simulation to its muse. Random numbers are generated with the help of `numpy` and the dataframe is provided by `pandas`.

__NOTE:__ In its current state, the simulation only accounts for single bets. The payout chart is listed but not in an implementable way.

In [1]:
import numpy as np
import pandas as pd

After the required imports, we should first define the wheels.
Below they are set for each of the three popular roulette games: 
the French (or European) version,
the version you'll find in American casinos,
and the triple zero wheel used by Sands.

In [2]:
frenchWheel = ['0', '32', '15', '19', '4', '23', '2', '25', '17', '34', '6', '27', '13', '36', '11', '30', '8', '23', '10', '5', '24', '16', '33', '1', '20', '14', '31', '9', '22', '18', '29', '7', '28', '12', '35', '3', '26']
americanWheel = ['0', '28', '9', '26', '30', '11', '7', '20', '32', '17', '5', '22', '34', '15', '3', '24', '36', '13', '1', '00', '27', '10', '25', '29', '12', '8', '19', '31', '18', '6', '21', '33', '16', '4', '23', '35', '14', '2']
sandsWheel = ['0', '000', '00', '32', '15', '19', '4', '21', '2', '25', '17', '34', '6', '27', '13', '36', '11', '30', '8', '23', '10', '5', '24', '16', '33', '1', '20', '14', '31', '9', '22', '18', '29', '7', '28', '12', '35', '3', '26']

Also below are the definition of payouts for winning various types of bets on each wheel, but this is not yet implemented.

In [3]:
# If a player hits on a single in an American game, that player wins 35 times the bet

payouts = [  # Bet, Winning Spaces, Payout Multiplier (winnings = wager + wager*multiplier), Game Type
           ('0', ['0'], 35, 'All'),
           ('00', ['00'], 35, 'American'),
           ('000', ['000'], 35, 'Sands'),  # TODO: Check value
           ('Straight', ['1..36'], 35, 'All'),
           # Inside
           ('Row', ['0', '00'], 17, 'American'),
           ('Split', ['two adjoining numbers'], 17, 'All'),
           ('Street', ['3 horizontal numbers'], 11, 'All'),
           ('Square', ['4 numbers in a block'], 8, 'All'),
           ('Basket-Fr', ['0', '1', '2', '3'], 8, 'French'),
           ('Basket-Am', ['0', '00', '1', '2', '3'], 6, 'American'),
           ('Double Street', ['2 rows or 3 numbers'], 5, 'All'),
           # Outside
           ('Column', ['Every third number (1,4,7...)'], 2, 'All'),
           ('Dozen', ['12 continuous numbers (1-12, 13-24, 25-36)'], 2, 'All'),
           ('Snake', [1, 5, 9, 12, 14, 16, 19, 23, 27, 30, 32, 34], 2, "All"),
           ('Impiar (Odd)', ['1, 3, 5 ...35'], 1, 'All'),
           ('Pair (Even)', ['2, 4, 6 ...36'], 1, 'All'),
           ('Red', [32, 19, 21, 25, 34, 27, 36, 30, 23, 5, 16, 1, 14, 9, 18, 7, 12, 3], 1, "All"),
           ('Black', [15, 4, 2, 17, 6, 13, 11, 8, 10, 24, 33, 20, 31, 22, 29, 28, 35, 26], 1, "All"),
           ('Manque (Low)', ['1-18'], 1, "All"),
           ('Passe (High)', ['19-36'], 1, "All"),
           ]  # TODO: break Winning Spaces into usable field


Then we should define the rules of the game. In all roulette variations, the croupier spins the wheel in one direction and sends the ball around the outside track going the opposite way.
This creates randomness in which pocket the ball will finally come to rest. 
We simulate this by choosing a randome pocket on the wheel when the `spin()` method is applied.
Breaking out this into a callable method allows us to simulate multiple plays when we actually begin the simulation.

To simulate the playing of various games, we'll create a base class and three variations that pull in different wheels.

In [4]:
class RouletteGame():
    def __init__(self):
        self.pockets = ['0', '1']  # French 37, American 38, Sands 39
        # self.pockets = americanWheel
        self.hitPocket = None
        self.possibleBets = []  # TODO: Set to accept other bets, only accepts Single bets currently

    def spin(self):
        self.hitPocket = np.random.choice(self.pockets)

    def getResult(self, bet, wager):
        if str(bet) == str(self.hitPocket):  # TODO: only set up for single bets right now
            # return wager * 35
            return wager * (35 + 1)
        return bet * -1

    def __str__(self):
        return 'Base Game'


class FrenchRoulette(RouletteGame):
    def __init__(self):
        self.pockets = frenchWheel

    def __str__(self):
        return 'French Roulette'


class AmericanRoulette(RouletteGame):
    def __init__(self):
        self.pockets = americanWheel

    def __str__(self):
        return 'American Roulette'


class SandsRoulette(RouletteGame):
    def __init__(self):
        self.pockets = sandsWheel

    def __str__(self):
        return 'Sands Roulette'

What's left is to create a method to actually play the game.
Here we take in the player choices of the type of game (game: French, American, or Sands), the number of times it will be simulated (plays), the pocket that will win if the ball lands in it (bet), and the amount of money to be lost if it doesn't. We'll play the game and capture the total amount won or lost in the `totalWinnings` variable. We'll then frame up some results with `pandas` and return them.

In [5]:
def playGame(game, plays, bet, wager):
    totalWinnings = 0
    for i in range(plays):
        game.spin()
        totalWinnings += game.getResult(bet, wager)
    df = pd.DataFrame({
                'Game':[game.__str__()],
                'Plays':[plays],
                'Bet':[bet],
                'Wager':[wager],
                'Total Wagered':[wager*plays],
                'Winnings':[totalWinnings - wager*plays],
                '% Winnings':[(totalWinnings - wager*plays)/(wager*plays) * 100],
                'Take Home':[totalWinnings],
                '% of Wagered':[totalWinnings/(wager*plays) * 100]
            })

    return df

Now we begin the simulation. 

In [6]:
game1 = playGame(
    FrenchRoulette(),
    100_000,  # Plays
    1,  # Number bet on
    10   # Amount wagered per play
    )

The result was put in a variable so I could adjust the formatting of the results for clarity.
As you can see, the payout for a single bet being 35:1 does give the edge to the casino on wheels that have 37 (including 0) pockets.

In [7]:
game1.style.format({
    'Plays': '{:,}',
    'Bet': lambda x:str(x).lower(),
    'Wager': '${:,.0f}',
    'Total Wagered': '${:,.0f}',
    'Winnings': '${:,.0f}',
    '% Winnings': '{:0.2f}%',
    'Take Home': '${:,.0f}',
    '% of Wagered': '{:.2f}%'
})

,Game,Plays,Bet,Wager,Total Wagered,Winnings,% Winnings,Take Home,% of Wagered
0,French Roulette,"100,000",1,$10,"$1,000,000","$-111,221",-11.12%,"$888,779",88.88%


Here are two more using the American and Sands tables.
They are condensed and printed vertically without the numbers formatted.

In [8]:
print(playGame(
    AmericanRoulette(),
    100_000,  # Plays
    1,  # Number bet on
    10   # Amount wagered per play
    ).transpose()
  )

                               0
Game           American Roulette
Plays                     100000
Bet                            1
Wager                         10
Total Wagered            1000000
Winnings                 -172591
% Winnings              -17.2591
Take Home                 827409
% of Wagered             82.7409


In [9]:
print(playGame(
    SandsRoulette(),
    100_000,  # Plays
    1,  # Number bet on
    10   # Amount wagered per play
    ).transpose()
  )

                            0
Game           Sands Roulette
Plays                  100000
Bet                         1
Wager                      10
Total Wagered         1000000
Winnings              -181977
% Winnings           -18.1977
Take Home              818023
% of Wagered          81.8023
